In [ ]:
import torch
import model_relu_deep_skip as model
import os
from torch.utils.mobile_optimizer import optimize_for_mobile
from Tikki import Tikki
from player import AI,random_player

In [ ]:
# Load model
n_players = 2
round_n = 5
card_n = 5
n_bets = 3
n_inputs = card_n + n_players*(1+round_n) + n_players
n_outputs = 52+3
hidden_size = 80
Actor = model.Actor(n_inputs,hidden_size,n_outputs,60)

script_dir = os.path.dirname(os.path.abspath(__file__))
model_folder_path = os.path.join(script_dir,r"model/trained_actor_V1-0.pt")

file_name = os.path.join(model_folder_path)
loaded = torch.load(file_name)
Actor.load_state_dict(loaded["model_state"])

In [ ]:
# Generate sample data
P1 = AI("P1",Actor,"cpu")
P2 = random_player("P2")
game = Tikki(players = [P1,P2])
game.new_game()
states  = []
for _ in range(10):
    while 1:
        # Or compute logprob, value directly with pass to net
        _, state, _, done,_ = game.step(P1)
        states.append(state)

        if done:
            break
#states = torch.stack(states).squeeze()
traced_script_module = torch.jit.trace(Actor, state)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_lite_interpreter("model/mobile_model_final_hard.pt")